# Inverse Kinematics
Inverse kinematics is a problem in robotics where we want to find the joint positions (theta) of a robot given a desired end-effector configuration (position and orientation). In other words, we want to determine how to position the robot's joints so that its end-effector (such as the robot's hand) reaches a specific point in space with a certain orientation.

$$
\begin{split}
\text{Forward Kinematics:   Given } \theta \text{, find } x \in SE(3) \\
\text{Inverse Kinematics:   Given } x \text{, find } \theta \text{ s.t. } T(\theta) = x
\end{split}
$$

### Challenges in Inverse Kinematics:
Unlike forward kinematics, which has a unique solution for a given set of joint values, the inverse kinematics problem can have multiple solutions or even no solutions at all. This is because there may be several ways to reach the desired end-effector position, and some configurations may be physically impossible for the robot to achieve.


### Approaches to Solving Inverse Kinematics:

1. Analytic Solutions:
In some cases, we can find closed-form solutions to the inverse kinematics problem using mathematical equations. These analytic solutions take advantage of geometric insights into the robot's structure. They are efficient and provide a direct formula to compute the joint angles. However, they may not exist for all robot configurations and can be complex to derive for more complex robots.
2. Numerical Solutions:
When analytic solutions are not feasible, we turn to numerical methods. Numerical inverse kinematics uses iterative techniques to find an approximate solution. It starts with an initial guess for the joint positions and iteratively adjusts them until the end-effector configuration converges to the desired one. Numerical methods are more versatile and can handle a wide range of robot configurations, but they may not find all possible solutions and require an initial guess.

### Planar 2R Robot Example:
To illustrate the analytic solution, a planar 2R robot is used as an example. This robot has two revolute joints and operates in a 2D plane. Given a desired end-effector position (x, y), the goal is to find the joint angles theta_1 and theta_2 that achieve this position.

$$
\begin{split}
\text{two-argument arctangent: } atan2(y, x) \in (-\pi, \pi] \\
\text{law of cosines: } c^2 = a^2 + b^2 - 2ab cos(\theta)
\end{split}
$$

The solution involves using the atan2 function to determine the angle of a vector from the origin to the point relative to the x-axis. Additionally, the law of cosines is used to relate the sides and angles of a triangle formed by the robot's links.

Based on the requested tip position:

- If the point is outside the robot's reachable workspace, there are no solutions.
- If the point is on the boundary of the workspace, there is one solution for theta_2 (either pi or 0).
- If the point is inside the workspace, there are two solutions for theta_2, which can be calculated using atan2 and the law of cosines.

$$
\begin{split}
\text{soultion 1: } \theta_1 = \gamma - \alpha, \theta_2 = \pi - \beta \\
\text{soultion 2: } \theta_1 = \gamma + \alpha, \theta_2 = \pi + \beta
\end{split}
$$

### Implications of Numerical Solutions:
While analytic solutions are straightforward and elegant when available, numerical methods are more widely applicable, especially for complex robots. They require an initial guess and converge to a single solution. However, they may not find all possible solutions and are not as efficient as analytic methods for robots with straightforward solutions.

## Numerical Inverse Kinematics
Forward kinematics is the process of determining the end-effector's configuration (position and orientation) based on the robot's joint values (theta). In contrast, inverse kinematics aims to find the joint values (theta) that achieve a desired end-effector configuration.

$$
FK : \theta \rightarrow T(\theta) \in SE(3)
$$
$$
coord FK : \theta \rightarrow f(\theta) \in R^m
$$
$$
coord IK : find \theta_d \text{ such that } x_d - f(\theta_d) = 0
$$

### Coordinate-Based Forward Kinematics
In this approach, instead of using transformation matrices, we work with a minimal set of coordinates (denoted as f(theta)) that describe the end-effector configuration. The goal of inverse kinematics is then to find joint values (theta_d) such that the difference between the desired end-effector configuration (x_d) and f(theta_d) is zero.

### Newton-Raphson Method for Inverse Kinematics
The Newton-Raphson method is a numerical root-finding technique used to solve nonlinear equations. In the context of inverse kinematics, it helps find joint values that satisfy the equation x_d - f(theta_d) = 0. The method involves iterative refinement of a initial guess (theta_zero) to converge towards a solution (theta_d).

### Illustrating the Newton-Raphson Method
Imagine plotting the function x_d - f(theta) as a function of theta. The roots of this function correspond to solutions of the inverse kinematics problem. The Newton-Raphson method starts with an initial guess (theta_zero), calculates the slope of the function at that point, and extends the slope to find a new guess (theta_1). This process is repeated iteratively until the sequence of guesses converges to the solution (theta_d).

### Generalization to Vectors
For multi-dimensional problems involving vectors of joint values and endpoint coordinates, we use the Taylor expansion of the function f(theta) around theta_d. This leads to an equation involving the Jacobian matrix (J(theta_i)) and the change in joint values (delta-theta). If J is invertible, we can solve for delta-theta directly using the pseudoinverse of J. The pseudoinverse has the advantage of finding solutions with the smallest joint value changes.

$$
x_d = f(\theta_d) = f(\theta^{(i)}) + \frac{\partial f}{\partial \theta} \bigg|_{\theta^{(i)}} (\theta_d - \theta^{i}) + O(\Delta \theta^{(i)})^2
$$
if we ignore the higher order terms, we get:
$$
\begin{align}
x_d - f(\theta^{(i)}) \approx J(\theta^{(i)}) \Delta \theta
\end{align}
$$
we can solve for the change in joint values:
$$
\Delta \theta = J^{-1}(\theta^{(i)}) (x_d - f(\theta^{(i)}))
$$
now rewrite the equation 1 we try to solve instead of prmultiplied by J_inverse, we can pre-multiply by the pseudoinverse of J:
$$
\begin{split}  
\Delta \theta &= J^{\dagger}(\theta^{(i)}) (x_d - f(\theta^{(i)})) \\
\theta^{(i+1)} &= \theta^{(i)} + \Delta \theta
\end{split}
$$
$J^{\dagger}$ is the pseudoinverse of J. The pseudoinverse has the advantage of finding solutions with the smallest joint value changes. It has the following properties:
- If there exists a $ \Delta \theta $ satifiying 1, then $ \Delta \theta^*$ has the smallest 2-norm among all solutions.
- If there no $ \Delta \theta $ satifiying 1, then $ \Delta \theta^*$ comes closest in the 2-norm sense.
### Numerical Algorithm
The Newton-Raphson numerical inverse kinematics algorithm involves the following steps:
1. Start with an initial guess (theta_zero).
2. Calculate the end-effector error (e = x_d - f(theta)).
3. If the error is small enough, the guess is the solution (theta_d).
4. If not, update the guess using the pseudoinverse of J times the error (theta_new = theta + J_pseudo * e) and repeat the process.  

Given $ x_d \in R^m $, $ \theta^{(0)} \in R^n $, set $ i = 0 $.  
Set $ e = x_d - f(\theta^{(i)}) $. While $ ||e|| > \epsilon $, for some small $ \epsilon $ do:
- Set $ \theta^{(i+1)} = \theta^{(i)} + J^{\dagger}(\theta^{(i)}) e $
- Set $ i = i + 1 $
### Application in Robot Control
This algorithm is used within a robot controller. At each time step, an updated desired end-effector configuration (x_d) is provided to the controller. The algorithm calculates an appropriate joint vector using the Newton-Raphson method, with the previous joint vector serving as a good initial guess. This approach enables the robot to achieve the desired configurations effectively.